In [1]:
import pymongo
import json

In [2]:
config_file = open("../config.cfg")
config = json.load(config_file)
client = pymongo.MongoClient(config["mongodb_uri"])
db = client[config["database"]]

In [3]:
collection = db["rajya_sabha_archive_questions"]

In [7]:
collection2 = db["rajya_sabha_snapshot"]

In [52]:
pipeline = [ {"$match": {}}, 
             {"$out": "rajya_sabha_snapshot"},
]
collection.aggregate(pipeline)

In [4]:
for row in collection.find({"qref":{"$regex":'.*\s'}}):
    qref = row["qref"].strip()
    print(qref)
#     collection.update_one({"_id":row["_id"]},{"$set":{"qref":qref}})

In [4]:
qrefs = list(collection.find({"date":""},{"qref":1}))

In [5]:
qrefs = [_["qref"] for _ in qrefs]

In [11]:
collection.count_documents({"date":""})

77162

In [7]:
qrefs_sessions = list(set([_.split("_")[1] for _ in qrefs]))

In [8]:
qrefs_sessions.sort()

In [9]:
qref_misses = {}
for _ in qrefs:
    session = _.split("_")[1]
    if session not in qref_misses.keys():
        qref_misses[session] = {}
    q_type = _.split("_")[-1]
    if q_type not in qref_misses[session].keys():
        qref_misses[session][q_type] = []
    q_no = _.split("_")[2]
    qref_misses[session][q_type].append(q_no)    

In [10]:
json.dump(qref_misses,open("missed_questions.json","w"))

In [15]:
qref_misses["237"]["UNSTARRED"]

['2743',
 '2879',
 '2876',
 '2872',
 '2878',
 '2880',
 '2868',
 '2877',
 '2874',
 '2864',
 '2863',
 '2871',
 '2873',
 '2859',
 '2860',
 '2870',
 '2875',
 '2866',
 '2867',
 '2855',
 '2856',
 '2865',
 '2869',
 '2852',
 '2862',
 '2851',
 '2861',
 '2858',
 '2848',
 '2847',
 '2857',
 '2854',
 '2853',
 '2843',
 '2850',
 '2839',
 '2838',
 '2849',
 '2835',
 '2834',
 '2845',
 '2846',
 '2844',
 '2842',
 '2841',
 '2830',
 '2837',
 '2840',
 '2827',
 '2836',
 '2826',
 '2823',
 '2822',
 '2833',
 '2831',
 '2819',
 '2832',
 '2829',
 '2818',
 '2816',
 '2828',
 '2824',
 '2825',
 '2814',
 '2812',
 '2821',
 '2820',
 '2810',
 '2808',
 '2806',
 '2817',
 '2815',
 '2803',
 '2813',
 '2811',
 '2799',
 '2809',
 '2807',
 '2795',
 '2805',
 '2804',
 '2801',
 '2802',
 '2791',
 '2800',
 '2793',
 '2789',
 '2798',
 '2787',
 '2797',
 '2786',
 '2794',
 '2796',
 '2783',
 '2782',
 '2790',
 '2778',
 '2788',
 '2792',
 '2766',
 '2784',
 '2776',
 '2772',
 '2781',
 '2785',
 '2770',
 '2780',
 '2768',
 '2779',
 '2775',
 '2777',
 

In [70]:
a = collection.aggregate(
    [ {"$group": { 
        "_id": { "qref": "$qref"},
        "count": { "$sum":  1 },
        "docs": { "$push": "$_id" }
    }},
    { "$match": {
        "count": { "$gt" : 1 }
    }}]
)

In [71]:
dups = list(a)

In [79]:
for dup in dups:
    for doc in dup['docs']:
        collection.delete_one({"_id":doc})